In [1]:
from dsoclasses.rinex.doris.rinex import DorisRinex
from os.path import join

# set the filename ...
data_path = "/home/xanthos/Software/AcademicSoftware/data"
drinex = join(data_path, "s6arx24001.001")

# construct the instance ...
rnx = DorisRinex(drinex)

## What is the time scale of DORIS (RINEX files) ?

In [2]:
print(f'RINEX reported time system is \"{rnx.time_sys}\"')

RINEX reported time system is "DOR"


Time tags in the RINEX file are reported in 'DORIS' time scale, which is aligned to TAI. Hence, in practice, we consider the RINEX tags to be in (a realization of) TAI.

## What are the nominal DORIS frequencies ?

DORIS transmits on two fundamental frequencies:
 - S1 at 2036.25 MHz ($\approx 15$cm), and
 - U2 at 401.25 ($\approx 75$cm) MHz

Beacon frequencies can be different than the nomial and depend on a "shift factor" $k$, reported in the RINEX file(s).
```text
D37  TLSB TOULOUSE                      10003S005  4   0    STATION REFERENCE   
D38  GR4B GRASSE                        10002S019  4 -15    STATION REFERENCE   
D39  BETB BETIO                         50305S001  3   0    STATION REFERENCE   
D40  NOXC NOUMEA                        92701S005  4   0    STATION REFERENCE   
D41  OWFC OWENGA                        50253S002  4   0    STATION REFERENCE   
D42  ASEB ASCENSION                     30602S005  3   0    STATION REFERENCE   
D43  TSTC TEST-FERMAT                   10003S006  4 -22    STATION REFERENCE 
```

To compute a beacon's nominal frequency, we use the formulae:
$$
    S1(k) = 543 \times USO_{F0} \times 0.75e0 + 543 \times USO_{F0} \times 87 \times k / (5 \times 2^26)
$$
and
$$
    U2(k) = 107 \times USO_{F0} \times 0.75 + 107 \times USO_{F0} \times 87 \times k / (5 \times 2^26)
$$
where $USO_{F0} = 5\times 10^6$; note that for $k=0$ the above results in the fundamental frequencies S1 and U2.

The above computation can be performed by loading:
```python
from dsoclasses.doris.algorithms import beacon_nominal_frequency
s1,u2 = beacon_nominal_frequency(k)
```

Given the RINEX above, what would the transmitted frequencies be for TLSB, GR4B and TSTC ?

In [3]:
from dsoclasses.doris.algorithms import beacon_nominal_frequency

s1,u2 = beacon_nominal_frequency(0)
print(f'TLSB nominal frequencies: {s1*1e-6:.2f} and {u2*1e-6:.2f} MHz')
s1,u2 = beacon_nominal_frequency(-15)
print(f'GR4B nominal frequencies: {s1*1e-6:.2f} and {u2*1e-6:.2f} MHz')
s1,u2 = beacon_nominal_frequency(-22)
print(f'TSTC nominal frequencies: {s1*1e-6:.2f} and {u2*1e-6:.2f} MHz')

TLSB nominal frequencies: 2036.25 and 401.25 MHz
GR4B nominal frequencies: 2036.24 and 401.25 MHz
TSTC nominal frequencies: 2036.23 and 401.25 MHz


## What does each beacon contribute to the observations ?

Let's extract e.g. C1 observations that are performed w.r.t the beacon DIOB:

In [4]:
# get the internal id of the 'DIOB' beacon:
site_id = rnx.name2id('DIOB')

# store measurements here (time, C1)
t = []; C1 = [];

# for every block in RINEX ...
for block in rnx:
    # for every beacon in the (current) data block ...
    for beacon, data in block:
        # match the site
        if beacon == site_id:
            # got DIOB! store the data
            t.append(block.t())
            C1.append(data['C1']['value'])

